# Stanza Tutorial

(C) 2023-2024 by [Damir Cavar](http://damir.cavar.me/)

**Version:** 1.1, January 2024

**Download:** This and various other Jupyter notebooks are available from my [GitHub repo](https://github.com/dcavar/python-tutorial-for-ipython).

**Prerequisites:**

In [ ]:
!pip install -U stanza

To install [spaCy](https://spacy.io/) follow the instructions on the [Install spaCy page](https://spacy.io/usage).

In [ ]:
!pip install -U pip setuptools wheel

The following installation of spaCy is ideal for my environment, i.e., using a GPU and CUDA 12.x. See the [spaCy homepage](https://spacy.io/usage) for detailed installation instructions.

In [ ]:
!pip install -U 'spacy[cuda12x,transformers,lookups,ja]'

## Introduction

This is a tutorial related to the [L645 Advanced Natural Language Processing](http://damir.cavar.me/l645/) course in Fall 2023 at Indiana University. The following tutorial assumes that you are using a newer distribution of [Python 3.x](https://python.org/) and [Stanza](https://stanfordnlp.github.io/stanza/) 1.5.1 or newer.

This notebook assumes that you have set up [Stanza](https://stanfordnlp.github.io/stanza/) on your computer with your [Python](https://python.org/) distribution. Follow the instructions on the [Stanza](https://stanfordnlp.github.io/stanza/) installation page to set up a working environment for the following code. The code will also require that you are online and that the specific language models can be downloaded and installed.

Loading the [Stanza](https://stanfordnlp.github.io/stanza/) module and [spaCy's Displacy](https://spacy.io/usage/visualizers) for visualization:

In [1]:
import stanza
from stanza.models.common.doc import Document
from stanza.pipeline.core import Pipeline
from spacy import displacy

c:\Users\damir\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


The following code will load the English language model for [Stanza](https://stanfordnlp.github.io/stanza/):

In [4]:
stanza.download('de')

2024-11-11 19:12:37 INFO: Downloaded file to C:\Users\damir\stanza_resources\resources.json
2024-11-11 19:12:37 INFO: Downloading default packages for language: de (German) ...


2024-11-11 19:13:10 INFO: Downloaded file to C:\Users\damir\stanza_resources\de\default.zip
2024-11-11 19:13:17 INFO: Finished downloading models and saved to C:\Users\damir\stanza_resources


We can configure the [Stanza](https://stanfordnlp.github.io/stanza/) pipeline to contain all desired linguistic annotation modules. In this case we use:
- tokenizer
- multi-word-tokenizer
- Part-of-Speech tagger
- lemmatizer
- dependency parser
- constituent parser

In [5]:
nlp = stanza.Pipeline('de', processors='tokenize,mwt,pos,lemma,ner,depparse,constituency,sentiment', package={"ner": ["ncbi_disease", "ontonotes"]}, use_gpu=False, download_method="reuse_resources")

2024-11-11 19:13:17 WARNING: Can not find ner: ncbi_disease from official model list. Ignoring it.
2024-11-11 19:13:17 WARNING: Can not find ner: ontonotes from official model list. Ignoring it.
2024-11-11 19:13:18 INFO: Loading these models for language: de (German):
| Processor    | Package      |
-------------------------------
| tokenize     | gsd          |
| mwt          | gsd          |
| pos          | gsd_charlm   |
| lemma        | gsd_nocharlm |
| constituency | spmrl_charlm |
| depparse     | gsd_charlm   |
| sentiment    | sb10k_charlm |

2024-11-11 19:13:18 INFO: Using device: cpu
2024-11-11 19:13:18 INFO: Loading: tokenize
2024-11-11 19:13:18 INFO: Loading: mwt
2024-11-11 19:13:18 INFO: Loading: pos
2024-11-11 19:13:18 INFO: Loading: lemma
2024-11-11 19:13:18 INFO: Loading: constituency
2024-11-11 19:13:19 INFO: Loading: depparse
2024-11-11 19:13:20 INFO: Loading: sentiment
2024-11-11 19:13:20 INFO: Done loading processors!


In [6]:
doc = nlp("Gummibärchen habe ich grüne noch keine gegessen.")
for i, sentence in enumerate(doc.sentences):
    print(f'====== Sentence {i+1} tokens =======')
    print(*[f'id: {token.id}\ttext: {token.text}' for token in sentence.tokens], sep='\n')

====== Sentence 1 tokens =======
id: (1,)	text: Gummibärchen
id: (2,)	text: habe
id: (3,)	text: ich
id: (4,)	text: grüne
id: (5,)	text: noch
id: (6,)	text: keine
id: (7,)	text: gegessen
id: (8,)	text: .


In [7]:
print(*[f'word: {word.text}\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}' for sent in doc.sentences for word in sent.words], sep='\n')

word: Gummibärchen	upos: NOUN	xpos: NN	feats: Case=Acc|Gender=Fem|Number=Plur
word: habe	upos: AUX	xpos: VAFIN	feats: Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin
word: ich	upos: PRON	xpos: PPER	feats: Case=Nom|Number=Sing|Person=1|PronType=Prs
word: grüne	upos: NOUN	xpos: NN	feats: Case=Acc|Gender=Fem|Number=Sing
word: noch	upos: ADV	xpos: ADV	feats: _
word: keine	upos: DET	xpos: PIS	feats: Case=Acc|Gender=Fem|Number=Sing|PronType=Neg
word: gegessen	upos: VERB	xpos: VVPP	feats: VerbForm=Part
word: .	upos: PUNCT	xpos: $.	feats: _


In [8]:
print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')

word: Gummibärchen 	lemma: Gummibärchen
word: habe 	lemma: haben
word: ich 	lemma: ich
word: grüne 	lemma: grün
word: noch 	lemma: noch
word: keine 	lemma: kein
word: gegessen 	lemma: essen
word: . 	lemma: .


In [9]:
for sentence in doc.sentences:
    print(sentence.constituency)

(ROOT (VROOT (S (NOUN Gummibärchen) (AUX habe) (PRON ich) (VP (NOUN grüne) (AP (ADV noch) (DET keine)) (VERB gegessen))) (PUNCT .)))


In [31]:
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')

entity: arthritis	type: DISEASE


In [10]:
print(*[f'token: {token.text}\tner: {token.ner}' for sent in doc.sentences for token in sent.tokens], sep='\n')

token: Gummibärchen	ner: None
token: habe	ner: None
token: ich	ner: None
token: grüne	ner: None
token: noch	ner: None
token: keine	ner: None
token: gegessen	ner: None
token: .	ner: None


In [11]:
for i, sentence in enumerate(doc.sentences):
    print("%d -> %d" % (i, sentence.sentiment))

0 -> 1


## Language ID

In [38]:
stanza.download(lang="multilingual")
stanza.download(lang="en")
# stanza.download(lang="fr")
stanza.download(lang="de")

2024-09-25 14:20:36 INFO: Downloaded file to C:\Users\damir\stanza_resources\resources.json
2024-09-25 14:20:36 INFO: Downloading default packages for language: multilingual (multilingual) ...
2024-09-25 14:20:36 INFO: File exists: C:\Users\damir\stanza_resources\multilingual\default.zip
2024-09-25 14:20:36 INFO: Finished downloading models and saved to C:\Users\damir\stanza_resources


2024-09-25 14:20:37 INFO: Downloaded file to C:\Users\damir\stanza_resources\resources.json
2024-09-25 14:20:37 INFO: Downloading default packages for language: en (English) ...
2024-09-25 14:20:38 INFO: File exists: C:\Users\damir\stanza_resources\en\default.zip
2024-09-25 14:20:41 INFO: Finished downloading models and saved to C:\Users\damir\stanza_resources


2024-09-25 14:20:41 INFO: Downloaded file to C:\Users\damir\stanza_resources\resources.json
2024-09-25 14:20:41 INFO: Downloading default packages for language: de (German) ...
2024-09-25 14:20:43 INFO: File exists: C:\Users\damir\stanza_resources\de\default.zip
2024-09-25 14:20:47 INFO: Finished downloading models and saved to C:\Users\damir\stanza_resources


In [41]:
nlp = Pipeline(lang="multilingual", processors="langid")
docs = ["Hello world.", "Hallo, Welt!", "Ciao mondo!", "Hola mundo!"]
docs = [Document([], text=text) for text in docs]
nlp(docs)
print("\n".join(f"{doc.text}\t{doc.lang}" for doc in docs))

2024-09-25 14:22:30 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-09-25 14:22:30 INFO: Downloaded file to C:\Users\damir\stanza_resources\resources.json
2024-09-25 14:22:30 INFO: Loading these models for language: multilingual ():
| Processor | Package |
-----------------------
| langid    | ud      |

2024-09-25 14:22:30 INFO: Using device: cpu
2024-09-25 14:22:30 INFO: Loading: langid
2024-09-25 14:22:30 INFO: Done loading processors!


Hello world.	en
Hallo, Welt!	it
Ciao mondo!	it
Hola mundo!	kmr


## Processing Dependency Parse Trees

I wrote the following function to convert the [Stanza](https://stanfordnlp.github.io/stanza/) dependency tree data structure to a [spaCy's Displacy](https://spacy.io/usage/visualizers) compatible data structure for the visualization of dependency trees using [spaCy's](https://spacy.io/) excellent visualizer:

In [12]:
def get_stanza_dep_displacy_manual(doc):
    res = []
    for x in doc.sentences:
        words = []
        arcs  = []
        for w in x.words:
            if w.head > 0:
                head_text = x.words[w.head-1].text
            else:
                head_text = "root"
            words.append({"text": w.text, "tag": w.upos})
            if w.deprel == "root": continue
            start = w.head-1
            end = w.id-1
            if start < end:
                arcs.append({ "start":start, "end":end, "label": w.deprel, "dir":"right"})
            else:
                arcs.append({ "start":end, "end":start, "label": w.deprel, "dir":"left"})
        res.append( { "words": words, "arcs": arcs } )
    return res

We can generate an annotation object with [Stanza](https://stanfordnlp.github.io/stanza/) similarly to [spaCy's](https://spacy.io/) approach submitting a sentence or text segment to the NLP pipeline we specified above and assigned to the `nlp` variable:

In [13]:
doc = nlp("Gummibärchen habe ich grüne noch keine gegessen.")

We can now generate the [spaCy](https://spacy.io/)-compatible data format from the dependency tree to be able to visualize it:

In [14]:
res = get_stanza_dep_displacy_manual(doc)

The rendering can be achieved using the [Displacy](https://spacy.io/usage/visualizers) call:

In [15]:
displacy.render(res, style="dep") # , manual=True, options={"compact":False, "distance":110})

AttributeError: 'dict' object has no attribute 'vocab'

## Data Format - CoNLL

In [47]:
from stanza.utils.conll import CoNLL

In [49]:
CoNLL.write_doc2conll(doc, "output.conllu")

**(C) 2023-2024 by [Damir Cavar](http://damir.cavar.me/) <<dcavar@iu.edu>>**